In [7]:
from cnns.core import LasagneGoogLeNetInceptionV3 as inceptv3
import lasagne
import numpy as np
import pickle as pkl
import theano
import theano.tensor as T
import skimage.io as io
from cnns.utils import cnn_utils as cu
import sys
import os
sys.path.insert(1, os.environ['CAFFE_HOME'] + '/python')
import caffe
import urllib2
from skimage import img_as_float
from PIL import Image
import StringIO
from cnns.utils import imagenet_utils as imu
import glob

In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
results_dir = '/Users/babasarala/Desktop/evaluation/results/UECFOOD100'

In [20]:
# Combine all the pickle files
def combine_checkpoint_results(results_path):
    pattern = '%s/*.p' % (results_path)
    
    def merge_dicts(d1, d2):
        d2_mod = {mem_id: cu.combine_probabilities(
            [(tup[0], tup[1]) for tup in tup_list]) for mem_id, tup_list in d2.iteritems()}
        assert len(set(d2_mod).intersection(set(d1))) == 0
        d1.update(d2_mod)

    merged_mem_cats = {}
    for f in glob.glob(pattern):
        merge_dicts(merged_mem_cats, pkl.load(open(f, "rb")))

    return merged_mem_cats

In [23]:
pred_labels = combine_checkpoint_results(results_dir)

In [31]:
N = len(pred_labels)
label_counts = {}
for _, pred_label in pred_labels.iteritems():
    l,p = max(pred_label, key=lambda x: x[1])
    if l not in label_counts:
        label_counts[l] = 0
    label_counts[l] += 1
for k in label_counts.keys():
    label_counts[k] /= 1.*N

In [32]:
label_counts

{u'devices': 7.874015748031496e-05,
 u'food': 0.6737007874015748,
 u'furry_friends': 7.874015748031496e-05,
 u'other': 0.3098425196850394,
 u'wildlife': 0.016299212598425195}